# Capstone Project The Battle of Neighborhoods (Week 2)



In [1]:
!conda install -c conda-forge geopy
!conda install -c conda-forge folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [2]:
import numpy as np # library to handle data in a vectorized manner
import time
import pandas as pd # library for data analsysis
# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

import json # library to handle JSON files
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import folium # map rendering library
from folium import plugins

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

import seaborn as sns

# import k-means from clustering stage
import os
from sklearn.cluster import KMeans



print('Libraries imported.')

Libraries imported.


In [3]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


In [4]:
# loading json data from file into a variable named newyork_data
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

neighborhoods_data = newyork_data['features']
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

# creating a dataframe with required columns
neighborhoods = pd.DataFrame(columns=column_names)

# populating data from newyork_data to a dataframe named neighborhoods
for data in neighborhoods_data:
    borough = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


# Report Content

<ol>
    <li>
        <h3>Introduction Section</h3> <br>
        Discussion of the business problem and the interested audience in this project.
    </li>
    <li> 
        <h3>Data Section</h3> <br>
        Description of the data that will be used to solve the problem and the sources.
    </li>
    <li> 
        <h3>Methodology section</h3> <br>
        Discussion and description of exploratory data analysis carried out,any inferential statistical testing performed, 
        and if any machine learnings were used establishing the strategy and purposes.
    </li>
    <li> 
        <h3>Results section</h3> <br>
        Discussion of the results.
    </li>
    <li> 
        <h3>Discussion section</h3> <br>
        Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
    </li>
    <li>
        <h3>Conclusion section - Report Conclusion.</h3>
    </li>
</ol>


## 1. Introduction Section: Discussion of the business problem and the audience who would be interested in this project.

### Description of the Problem and Background

### Scenario:

I am a data scientist residing in Lahore, Pakistan. I currently live within walking distance to a market and I enjoy many ammenities and venues in the area, such as various international cousine restaurants, cafes, food shops and entertainment. I have been offered a great opportunity to work for a leader firm in Manhattan, NY. I am very excited and I want to use this opportunity to practice my learnings in Coursera in order to answer relevant questions arisen. The key question is : How can I find a convenient and enjoyable place similar to mine now in Pakistan? Certainly, I can use available real estate apps and Google but the idea is to use and apply myself the learned tools during the course. In order to make a comparison and evaluation of the rental options in Manhattan NY, I must set some basis, therefore the apartment in Manhattan must meet the following demands:

<ul>
    <li>Apartment must be 2 or 3 bedrooms</li>
    <li>Desired location is near a metro station in the Manhattan area and within 1.0 mile (1.6 km) radius</li>
    <li>Price of rent must not exceed 7,000 per month</li>
    <li>Top ammenities in the selected neighborhood shall be similar to current residence</li>
    <li>Desirable to have venues such as coffee shops, restaurants, gym and food shops</li>
    <li>As a reference, I have included a map of venues near current residence in Tanzania.</li>
</ul>

### Business Problem:

The challenge is to find a suitable apartment for rent in Manhattan NY that complies with the demands on location, price and venues. The data required to resolve this challenge is described in the following section 2, below.

### Interested Audience

I believe this is a relevant challenge with valid questions for anyone moving to other large city in US, EU or Asia. The same methodology can be applied in accordance to demands as applicable. This case is also applicable for anyone interested in exploring starting or locating a new business in any city. Lastly, it can also serve as a good practical exercise to develop Data Science skills.

## 2. Data Section:

### Description of the data and its sources that will be used to solve the problem

I Currently reside in GECHS in the city of Lahore. I use Foursquare to identify the venues around the area of residence which are then shown in the Lahore, Link Road map shown in methodology and execution in section 3.0. It serves as a reference for comparison with the desired future location in Manhattan NY.

### Description of the Data

The following data is required to answer the issues of the problem:

<ul>
    <li>List of Boroughs and neighborhoods of Manhattan with their geodata (latitude and longitude)</li>
    <li>List of Subway metro stations in Manhattan with their address location</li>
    <li>List of apartments for rent in Manhattan area with their addresses and price</li>
    <li>Preferably, a list of apartment for rent with additional information, such as price, address, area, number of beds, etc</li>
    <li>Venues for each Manhattan neighborhood (than can be clustered)</li>
    <li>Venues for subway metro stations, as needed</li>
</ul>

## 2.2 Data Required to resolve the problem

In order to make a good choice of a similar apartment in Manhattan NY, the following data is required: List/Information on neighborhoods form Manhattan with their Geodata (latitude and longitude. List/Information about the subway metro stations in Manhattan with geodata. Listed apartments for rent in Manhattan area with descriptions (how many beds, price, location, address) Venues and Ammenities in the Manhattan neighborhoods (e.g. top 10) 2.3 sources and manipulation The list of Manhattan neighborhoods is worked out during Lab exercise during the course. A csv file was created which will be read in order to create a dataframe and its mapping. The csv file 'mh_neigh_data.csv' has the following below data structure. The file will be directly read to the Jupiter Notebook for convenience and space savings. The clustering of neighborhoods and mapping will be shown however. An algorithm was used to determine the geodata from Nominatim . The actual algorithm coding may be shown in 'markdown' mode becasues it takes time to run.

In [5]:
# Extracting Manhattan neighborhoods from new york data.
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


A list of Manhattan subway metro stops was complied in Numbers (Apple excel) and it was complemeted with wikipedia data ( https://en.wikipedia.org/wiki/List_of_New_York_City_Subway_stations_in_Manhattan) and information from NY Transit authority and Google maps (https://www.google.com/maps/search/manhattan+subway+metro+stations/@40.7837297,-74.1033043,11z/data=!3m1!4b1) for a final consolidated list of subway stops names and their address. The geolocation was obtained via an algorythm using Nominatim. Details will be shown in the execution of methodolody in section 3.0.

A list of places for rent was collected by web-browsing real estate companies in Manhattan: http://www.rentmanhattan.com/index.cfm?page=search&state=results, https://www.nestpick.com/search?city=new-york&page=1&order=relevance&district=manhattan&gclid=CjwKCAiAjNjgBRAgEiwAGLlf2hkP3A-cPxjZYkURqQEswQK2jKQEpv_MvKcrIhRWRzNkc_r-fGi0lxoCA7cQAvD_BwE&type=apartment&display=list https://www.realtor.com/apartments/Manhattan_NY A csv file was compiled with the rental place that indicated: areas of Manhattan, address, number of beds, area and monthly rental price. The csv file "nnnn.csv" had the following below structure. An algorithm was used to create all the geodata using Nominatim, as shown in section 3.0. The actual algorithm coding may be shown in 'markdown' mode becasues it takes time to run. With the use of geolocator = Nominatim() , it was possible to determine the latitude and longiude for the subway metro locations as well as for the geodata for each rental place listed. The loop algorithms used are shown in the execution of data in section 3.0 "Great_circle" function from geolocator was used to calculate distances between two points , as in the case to calculate average rent price for units around each subway station and at 1.6 km radius. Foursquare is used to find the avenues at Manhattan neighborhoods in general and a cluster is created to later be able to search for the venues depending of the location shown.

## 2.3 How the data will be used to solve the problem

The data will be used as follows:

<ul>
    <li>Use Foursquare and geopy data to map top 10 venues for all Manhattan neighborhoods and clustered in groups (as per Course LAB)</li>
    <li>Use foursquare and geopy data to map the location of subway metro stations, separately and on top of the above clustered map in order to be able to identify the venues and ammenities near each metro station, or explore each subway location separately.</li>
    <li>Use Foursquare and geopy data to map the location of rental places, in some form, linked to the subway locations.</li>
    <li>create a map that depicts, for instance, the average rental price per square ft, around a radious of 1.0 mile (1.6 km) around each subway station - or a similar metrics. I will be able to quickly point to the popups to know the relative price per subway area.</li>
    <li>Addresses from rental locations will be converted to geodata(lat,long) using Geopy-distance and Nominatim.</li>
    <li>Data will be searched in open data sources if available, from real estate sites if open to reading, libraries or other government agencies such as Metro New York MTA, etc.</li>
</ul>

## 2.4 The procesing of these DATA will allow to answer the key questions to make a decision:

<ul>
    <li>What is the cost of rent (per square ft) around a mile radius from each subway metro station?</li>
    <li>What is the area of Manhattan with best rental pricing that meets criteria established?</li>
    <li>What is the distance from work place ( Park Ave and 53 rd St) and the tentative future home?</li>
    <li>What are the venues of the two best places to live? How the prices compare?</li>
    <li>How venues distribute among Manhattan neighborhoods and around metro stations?</li>
    <li>Are there tradeoffs between size and price and location?</li>
    <li>Any other interesting statistical data findings of the real estate and overall data.</li>
</ul>

## 2.5 Mapping of Data

The following maps were created to facilitate the analysis and the choice of the palace to live.

<ul>
    <li>Manhattan map of Neighborhoods</li>
    <li>Manhattan subway metro locations</li>
    <li>Manhattan map of places for rent</li>
    <li>Manhattan map of clustered venues and neighborhoods</li>
    <li>Combined maps of Manhattan rent places with subway locations</li>
    <li>Combined maps of Manhattan rent places with subway locations and venues clusters</li>
</ul>

## 3. Methodology section

This section represents the main component of the report where the data is gathered, prepared for analysis. The tools described are used here and the Notebook cells indicates the execution of steps.

## The analysis and the stragegy:

The strategy is based on mapping the above described data in section 2.0, in order to facilitate the choice of at least two candidate places for rent.
The choice is made based on the demands imposed ie location near a subway, rental price and similar venues to Dar es salaam, Tanzania.
This visual approach and maps with popups labels allow quick identification of location, price and feature, thus making the selection very easy.

The procesing of these DATA and its mapping will allow to answer the key questions to make a decision:

<ul>
    <li>What is the cost of available rental places that meet the demands?</li>
    <li>What is the cost of rent around a mile radius from each subway metro station?</li>
    <li>What is the area of Manhattan with best rental pricing that meets criteria established?</li>
    <li>What is the distance from work place ( Park Ave and 53 rd St) and the tentative future rental home?</li>
    <li>What are the venues of the two best places to live? How the prices compare?</li>
    <li>How venues distribute among Manhattan neighborhoods and around metro stations?</li>
    <li>Are there tradeoffs between size and price and location?</li>
    <li>Any other interesting statistical data findings of the real estate and overall data.</li>
</ul>


# METHODOLOY EXECUTION - Mapping Data

## Link Road, Lahore Map - Current residence and venues in neighborhood for comparison to future Manhattan renting place

In [8]:
address = 'Link Road, Model Town, Lahore'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
home_latitude = location.latitude
home_longitude = location.longitude
print('The geograpical coordinate of Link Road, Model Town, Lahore are {}, {}.'.format(home_latitude, home_longitude))

The geograpical coordinate of Link Road, Model Town, Lahore are 31.4851048, 74.3262044.


In [9]:
# The code was removed by Watson Studio for sharing.

In [10]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    home_latitude, 
    home_longitude, 
    radius, 
    LIMIT)


# results display is hidden for report simplification 
results = requests.get(url).json()

### function that extracts the category of the venue - copied from the Foursquare lab.

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [12]:
venues = results['response']['groups'][0]['items']
nearby_venues_home = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

nearby_venues_home = nearby_venues_home.loc[:, filtered_columns]

# filter the category for each row
nearby_venues_home['venue.categories'] = nearby_venues_home.apply(get_category_type, axis=1)

# clean columns
nearby_venues_home.columns = [col.split(".")[-1] for col in nearby_venues_home.columns]

nearby_venues_home.shape

(13, 4)

In [13]:
nearby_venues_home.head()

,name,categories,lat,lng
0,Model Town Park,Park,31.484509,74.326320
1,Cafe Pizzario,Café,31.488059,74.325900
2,Bank Square Market,Market,31.488485,74.325950
3,Amjad Samosay,Snack Place,31.482855,74.330150
4,Gloria Jean's Coffees,Coffee Shop,31.488353,74.325995


### Map of Link Road, residence place with venues in Neighborhood - for reference

In [14]:
latitude = home_latitude
longitude = home_longitude

# create map of link road model town place  using latitude and longitude values
map_sg = folium.Map(location=[latitude, longitude], zoom_start=17)

# add markers to map
for lat, lng, label in zip(nearby_venues_home['lat'], nearby_venues_home['lng'], nearby_venues_home['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=30,
        radius=6,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.6,
    ).add_to(map_sg)  
    
map_sg

### MANHATTAN NEIGHBORHOODS - DATA AND MAPPING

Cluster neighborhood data was produced with Foursquare during course lab work. <br>
A csv file was produced containing the neighborhoods around the 40 Boroughs. <br>
Now, the csv file is just read for convenience and consolidation of report. <br>

In [15]:
# return the venues in given radius for all the names and coordinates passed in params
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [16]:
# Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.

manhattan_venues = getNearbyVenues(names=manhattan_data['Neighborhood'],
                                   latitudes=manhattan_data['Latitude'],
                                   longitudes=manhattan_data['Longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron
Hudson Yards


In [61]:
manhattan_onehot = pd.get_dummies(manhattan_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = manhattan_venues['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped.head()

,Neighborhood,Accessories Store,Adult Boutique,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,...,Video Store,Vietnamese Restaurant,Volleyball Court,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Battery Park City,0.0,0.0,0.0,0.000000,0.014706,0.0,0.0,0.00,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.014706,0.0,0.000000,0.000000
1,Carnegie Hill,0.0,0.0,0.0,0.000000,0.010526,0.0,0.0,0.00,0.010526,...,0.0,0.010526,0.0,0.0,0.0,0.010526,0.031579,0.0,0.010526,0.031579
2,Central Harlem,0.0,0.0,0.0,0.066667,0.044444,0.0,0.0,0.00,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000
3,Chelsea,0.0,0.0,0.0,0.000000,0.040000,0.0,0.0,0.01,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.010000,0.000000,0.0,0.010000,0.000000
4,Chinatown,0.0,0.0,0.0,0.000000,0.030000,0.0,0.0,0.00,0.000000,...,0.0,0.030000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.010000


In [62]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [66]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Boat or Ferry,Memorial Site,Food Court,Shopping Mall,Playground,BBQ Joint
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Bar,Bakery,Gym / Fitness Center,Gym,Italian Restaurant,Bookstore,Yoga Studio
2,Central Harlem,African Restaurant,Cosmetics Shop,Art Gallery,Chinese Restaurant,Seafood Restaurant,Bar,French Restaurant,American Restaurant,Food Truck,Caribbean Restaurant
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Café,Bakery,French Restaurant,Bar,Pizza Place,Market,Park
4,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Hotpot Restaurant,Dessert Shop,Spa,Bubble Tea Shop,American Restaurant,Optical Shop,Vietnamese Restaurant


In [67]:
# set number of clusters
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([3, 1, 0, 1, 0, 3, 3, 2, 0, 1], dtype=int32)

### Map of Manhattan neighborhoods with top 10 clustered venues

popus allow to identify each neighborhood and the cluster of venues around it in order to proceed to examine in more detail in the next cell

In [68]:
address = 'Manhattan, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
manhattan_latitude = location.latitude
manhattan_longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [81]:
manhattan_merged = neighborhoods_venues_sorted
manhattan_merged['Cluster Labels'] = kmeans.labels_
manhattan_merged = pd.merge(manhattan_merged, manhattan_data[['Neighborhood', 'Latitude', 'Longitude']], on="Neighborhood", how="left")
manhattan_merged.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
0,Battery Park City,Park,Hotel,Gym,Coffee Shop,Boat or Ferry,Memorial Site,Food Court,Shopping Mall,Playground,BBQ Joint,3,40.711932,-74.016869
1,Carnegie Hill,Coffee Shop,Café,Pizza Place,Bar,Bakery,Gym / Fitness Center,Gym,Italian Restaurant,Bookstore,Yoga Studio,1,40.782683,-73.953256
2,Central Harlem,African Restaurant,Cosmetics Shop,Art Gallery,Chinese Restaurant,Seafood Restaurant,Bar,French Restaurant,American Restaurant,Food Truck,Caribbean Restaurant,0,40.815976,-73.943211
3,Chelsea,Coffee Shop,Art Gallery,American Restaurant,Café,Bakery,French Restaurant,Bar,Pizza Place,Market,Park,1,40.744035,-74.003116
4,Chinatown,Chinese Restaurant,Cocktail Bar,Bakery,Hotpot Restaurant,Dessert Shop,Spa,Bubble Tea Shop,American Restaurant,Optical Shop,Vietnamese Restaurant,0,40.715618,-73.994279


In [82]:
# create map of Manhattan using latitude and longitude values from Nominatim
latitude = manhattan_latitude
longitude = manhattan_longitude

kclusters=5
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=20,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
  # add markers for rental places to map
for lat, lng, label in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_clusters)    
    
       
map_clusters



## Examine a paticular Cluster - print venues



### Cluster 1

In [83]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
2,African Restaurant,Seafood Restaurant,Bar,French Restaurant,American Restaurant,Food Truck,Caribbean Restaurant,0,40.815976,-73.943211
4,Chinese Restaurant,Dessert Shop,Spa,Bubble Tea Shop,American Restaurant,Optical Shop,Vietnamese Restaurant,0,40.715618,-73.994279
8,Bar,Cocktail Bar,Juice Bar,Wine Bar,Dessert Shop,Italian Restaurant,Vietnamese Restaurant,0,40.727847,-73.982226
11,Bar,Italian Restaurant,Mexican Restaurant,Cocktail Bar,Coffee Shop,Thai Restaurant,Diner,0,40.737210,-73.981376
20,Bar,Pizza Place,Mexican Restaurant,Dog Run,Bubble Tea Shop,Café,Caribbean Restaurant,0,40.797307,-73.964286
36,Italian Restaurant,Mediterranean Restaurant,Bakery,Pub,Sports Bar,Pizza Place,Thai Restaurant,0,40.787658,-73.977059
39,Italian Restaurant,Sushi Restaurant,Deli / Bodega,Pizza Place,Diner,Japanese Restaurant,Mexican Restaurant,0,40.775930,-73.947118


### Cluster 2

In [84]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
1,Coffee Shop,Bakery,Gym / Fitness Center,Gym,Italian Restaurant,Bookstore,Yoga Studio,1,40.782683,-73.953256
3,Coffee Shop,Bakery,French Restaurant,Bar,Pizza Place,Market,Park,1,40.744035,-74.003116
9,Coffee Shop,Hotel,Mexican Restaurant,Café,Bar,Gym,Gym / Fitness Center,1,40.707107,-74.010665
16,Coffee Shop,Café,Cocktail Bar,Deli / Bodega,Burger Joint,Gym,Gym / Fitness Center,1,40.768113,-73.958860
22,Gym,Big Box Store,Supplement Shop,Steakhouse,Miscellaneous Shop,Shopping Mall,Yoga Studio,1,40.876551,-73.910660
23,Coffee Shop,Theater,Pizza Place,Steakhouse,Bookstore,Spa,Sushi Restaurant,1,40.754691,-73.981669
24,Korean Restaurant,Cosmetics Shop,Gym / Fitness Center,Clothing Store,Coffee Shop,Bakery,Flower Shop,1,40.748510,-73.988713
26,Coffee Shop,Mediterranean Restaurant,Hotel,Pizza Place,Burger Joint,Gym / Fitness Center,Bakery,1,40.748303,-73.978332
27,Italian Restaurant,Wine Bar,Grocery Store,French Restaurant,Sandwich Place,Bookstore,Ice Cream Shop,1,40.723259,-73.988434
29,Italian Restaurant,French Restaurant,Bakery,Salon / Barbershop,Women's Store,Boutique,Wine Bar,1,40.722184,-74.000657


### Cluster 3

In [85]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
7,Mexican Restaurant,Deli / Bodega,Sandwich Place,Liquor Store,Cocktail Bar,Donut Shop,Beer Bar,2,40.792249,-73.944182
13,Pizza Place,Mexican Restaurant,Bakery,Cocktail Bar,Sandwich Place,Chinese Restaurant,School,2,40.823604,-73.949688
15,Mexican Restaurant,Park,Bakery,Frozen Yogurt Shop,Caribbean Restaurant,Chinese Restaurant,Spanish Restaurant,2,40.867684,-73.921210
18,Bakery,Ice Cream Shop,Coffee Shop,Mediterranean Restaurant,Cocktail Bar,Salon / Barbershop,Sandwich Place,2,40.719324,-73.997305
19,Chinese Restaurant,Pizza Place,Japanese Restaurant,Ramen Restaurant,Cocktail Bar,Yoga Studio,Clothing Store,2,40.717807,-73.980890
21,Coffee Shop,Mexican Restaurant,Italian Restaurant,Falafel Restaurant,Bike Trail,Sushi Restaurant,Lounge,2,40.816934,-73.957385
28,Park,Greek Restaurant,Coffee Shop,Soccer Field,Liquor Store,Sandwich Place,Scenic Lookout,2,40.762160,-73.949168
33,Café,Sushi Restaurant,Greek Restaurant,Garden,Spanish Restaurant,Dog Run,Coffee Shop,2,40.746917,-73.971219
37,Café,Coffee Shop,Mexican Restaurant,Latin American Restaurant,Tapas Restaurant,Spanish Restaurant,Italian Restaurant,2,40.851903,-73.936900


### Cluster 4

In [86]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
0,Park,Boat or Ferry,Memorial Site,Food Court,Shopping Mall,Playground,BBQ Joint,3,40.711932,-74.016869
5,Coffee Shop,Spa,Italian Restaurant,Yoga Studio,Café,French Restaurant,Park,3,40.715229,-74.005415
6,Italian Restaurant,Wine Shop,American Restaurant,Hotel,Sandwich Place,Cocktail Bar,Gym,3,40.759101,-73.996119
10,Gym / Fitness Center,Furniture / Home Store,Vegetarian / Vegan Restaurant,Spa,Mediterranean Restaurant,Café,Salon / Barbershop,3,40.739673,-73.990947
12,Italian Restaurant,Ice Cream Shop,American Restaurant,Sandwich Place,Cosmetics Shop,Bubble Tea Shop,Dessert Shop,3,40.726933,-73.999914
14,American Restaurant,Café,Coffee Shop,Dog Run,Gym,Park,Burger Joint,3,40.756658,-74.000111
17,Plaza,Performing Arts Venue,Theater,Wine Shop,American Restaurant,Italian Restaurant,Coffee Shop,3,40.773529,-73.985338
25,Park,Deli / Bodega,Sandwich Place,Burger Joint,Donut Shop,Grocery Store,Greek Restaurant,3,40.808000,-73.963896
31,Italian Restaurant,Coffee Shop,Gym,Pizza Place,Spa,Beer Garden,Beer Bar,3,40.760280,-73.963556
32,American Restaurant,Wine Bar,Spa,Coffee Shop,Skate Park,Hotel,Men's Store,3,40.721522,-74.010683


### Cluster 5

In [87]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,1st Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels,Latitude,Longitude
30,Boat or Ferry,Farmers Market,Skating Rink,Bistro,Gas Station,Baseball Field,Gym / Fitness Center,4,40.731,-73.974052


### After examining several cluster data , I concluded that cluster# 4 resembles closer Model Town, therefore providing guidance as to where to look for the future apartment

## Problem Resolution - Select the apartment for rent<br>
### After examining, I have chosen to look for an apartment in cluster 4 because it resembles very closely to my current location.